In [1]:
# https://github.com/tallesfelix/directions-with-osm-networkx/blob/master/Creating%20routes%20with%20networkx%20and%20ipyleaflet.ipynb
import networkx as nx
import osmnx as ox
from shapely.geometry import LineString, mapping
import geopandas as gpd
from ipyleaflet import *

/home/blaxeep/.local/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  warnings.warn(


In [2]:
place_name = "Greece"
ox.config(use_cache=True, log_console=True)
#cf = '["highway"~"motorway|motorway_link|trunk|secondary|primary"]'
cf = '["highway"~"motorway|motorway_link"]'
greece_graph = ox.graph_from_place(place_name, network_type='drive', custom_filter=cf)
#fig, ax = ox.plot_graph(graph)
greece_nodes, greece_edges = ox.graph_to_gdfs(greece_graph)

zogr = "Zografou, Greece"
ox.config(use_cache=True, log_console=True)
#cf = '["highway"~"motorway|motorway_link|trunk|secondary|primary"]'
#cf = '["highway"~"motorway|motorway_link|trunk|trunk_link|primary_link|primary"]'
zogr_graph = ox.graph_from_place(zogr, network_type='drive')
zogr_nodes, zogr_edges = ox.graph_to_gdfs(zogr_graph)

In [5]:
G3 = nx.compose(greece_graph, zogr_graph)

In [6]:
fig, ax = ox.plot_graph(G3)

In [11]:
nodes, edges = ox.graph_to_gdfs(G3)
center = (37.714155,24.056792)
m = Map(center=center, basemap=basemaps.CartoDB.Positron, zoom=8)

to_marker_style = AwesomeIcon(
    name='circle',
    icon_color='white',
    marker_color='red',
    spin=False
)
from_marker = Marker(location=center)
print(from_marker)
to_marker = Marker(location=center, icon=to_marker_style)

def set_nearest_node(marker):
    marker.nearest_node = ox.get_nearest_node(G3, marker.location)
    return

path_layer_list = []

def handle_change_location(event, marker):
    event_owner = event['owner']
    event_owner.nearest_node = ox.get_nearest_node(G3, event_owner.location)
    marker.neares_node = ox.get_nearest_node(G3, marker.location)
    
    shortest_path = nx.dijkstra_path(G3, event_owner.nearest_node, marker.neares_node, 
                                     weight='length')
    
    if len(path_layer_list) == 1:
        m.remove_layer(path_layer_list[0])
        path_layer_list.pop()
    
    shortest_path_points = nodes.loc[shortest_path]
    path = gpd.GeoDataFrame([LineString(shortest_path_points.geometry.values)], columns=['geometry'])
    path_layer = GeoData(geo_dataframe=path, style={'color':'black', 'weight':2})
    m.add_layer(path_layer)
    path_layer_list.append(path_layer)
    
    
    
from_marker.observe(lambda event: handle_change_location(event, to_marker), 'location')
to_marker.observe(lambda event: handle_change_location(event, from_marker), 'location')

m.add_layer(from_marker)
m.add_layer(to_marker)
set_nearest_node(from_marker)
set_nearest_node(to_marker)
m

Marker(location=[37.714155, 24.056792], options=['alt', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'rotation_angle', 'rotation_origin', 'title', 'z_index_offset'])


Map(center=[37.714155, 24.056792], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

In [15]:
supermarket = '["shop"~"supermarket"]'
G = ox.graph_from_place('Zografou, Greece', custom_filter=supermarket)

EmptyOverpassResponse: There are no data elements in the response JSON